gridspec-create latlon --helpconda create -n esmpy -c conda-forge esmpy## Regridding a cube sphere dataset to a regular lat/lon grid

This script uses the sparselt library available at https://github.com/LiamBindle/sparselt.git 

Jupyter notebook adapted from Liam Bindle's example: https://github.com/LiamBindle/sparselt/blob/master/examples/esmf_regrid_example.py

First navigate to a folder where you will store your regrid files ("/path/to/your/regrid/files/") and use these commands to generate ESMF weights file:
(note: for 2x2.5° try 91 144 (144 = 360/2.5))

```console
$ gridspec-create gcs 90 # change 90 to your input file cube sphere resolution (48, 90, etc)

$ gridspec-create latlon 180 288  # change 91 144 to your desired lat/lon 

$ ESMF_RegridWeightGen -s c24_gridspec.nc -d regular_lat_lon_91x144.nc -m conserve -w esmf_regrid_weights_c24_to_latlon91x144.nc --tilefile_path /path/to/your/regrid/files/ # you can change the filenames to reflect your input/output resolution

```

In [1]:
import os

import gcpy.constants as gcon
import sparselt.esmf
import sparselt.xr
import xarray as xr
from dask.diagnostics import ProgressBar
import gcpy
import numpy as np

skip_vars = gcon.skip_these_vars

In [38]:
# User changes
cube_res = '90_s2d00'#'180'
lat = 180
lon = 288

regrid_path = "/net/fs11/d0/emfreese/GCrundirs/IRF_runs/regrid_files/"
weights_file = f"esmf_regrid_weights_c{cube_res}_to_latlon{lat}x{lon}.nc"
reg_latlon = f"regular_lat_lon_{lat}x{lon}.nc"

month = 2

if month == 1 or 3 or 5 or 7 or 8 or 10 or 12:
    dates = np.arange(1,30)
elif month == 2:
    dates = np.arange(1, 28)
else:
    dates = np.arange(1,31)

filenames = [
    f"GEOSChem.SpeciesConc.20160{month}{str(date).zfill(2)}_0000z.nc4" for date in dates
    ] + [
    f"GEOSChem.AerosolMass.20160{month}{str(date).zfill(2)}_0000z.nc4" for date in dates
    ]+ [
    f"GEOSChem.Emissions.20160{month}{str(date).zfill(2)}_0000z.nc4" for date in dates
]


model_run = 'stretch_step'

location = 'Cambod'

#/net/fs11/d0/emfreese/GCrundirs/IRF_runs/RRTMG_base/template/OutputDir/
input_path = f'/net/fs11/d0/emfreese/GCrundirs/IRF_runs/{model_run}/{location}/OutputDir/' #"/path/to/your/cubesphere/input/files/"
destination_path = f'/net/fs11/d0/emfreese/GCrundirs/IRF_runs/{model_run}/{location}/mod_output/' #"/path/to/your/latlon/output/files/"

In [39]:
def main():
    for filename in filenames:
                    
        print("Opening cube-sphere data")
        ds = xr.open_dataset(
            input_path + filename, engine="netcdf4", drop_variables=skip_vars
        )
        ##lyssa added
        ds = ds.drop(labels = ['corner_lons','corner_lats'])
        ##finished lyssa added
        transform = sparselt.esmf.load_weights(
            regrid_path + weights_file,
            input_dims=[("nf", "Ydim", "Xdim"), (6, cube_res, cube_res)],
            output_dims=[("lat", "lon"), (lat, lon)],
        )

        output_template = xr.open_dataset(regrid_path + reg_latlon)

        print("Transforming data")
        with ProgressBar():
            ds_new = sparselt.xr.apply(transform, ds, output_template)
        
        print("Saving to NetCDF")
        output_file = os.path.join(destination_path, filename)
        with ProgressBar():
             ds_new.load().to_netcdf(output_file)

In [40]:
if __name__ == "__main__":
    main()

Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming data
Saving to NetCDF
Opening cube-sphere data
Transforming da